In [ ]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import time
from bs4 import BeautifulSoup
import asyncio
import aiohttp
from tqdm.asyncio import tqdm_asyncio
import nest_asyncio
import os



In [2]:
ingredients = [
    "rating", "almond", "apple", "apricot", "artichoke", "arugula", "asian pear", "asparagus", "avocado",
    "bacon", "banana", "barley", "basil", "beef", "beet", "bell pepper",
    "berry", "blackberry", "blue cheese", "blueberry", "bok choy", "broccoli", "broccoli rabe", "brussel sprout",
    "butter", "buttermilk", "butternut squash", "cabbage", "capers", "carrot", "cashew",
    "cauliflower", "celery", "cheese", "cherry", "chestnut", "chickpea", "chile", "chile pepper", "chili",
    "chive", "chocolate", "cilantro", "cinnamon", "clove", "coconut", "cod", "collard greens",
    "corn", "cornmeal", "cottage cheese", "crab", "cranberry", "cream cheese", "cucumber", "cumin", "currant", "curry",
    "date", "dill", "egg", "eggplant", "endive", "feta", "fig", "fish", "fontina",
    "garlic", "ginger", "goat cheese", "gouda", "grape", "grapefruit", "green bean",
    "green onion/scallion", "ground beef", "ground lamb", "guava", "hazelnut", "honey", "honeydew", "horseradish",
    "hot pepper", "jalapeño", "jam or jelly", "kale", "lamb",
    "lentil", "lettuce", "lima bean", "lime", "lobster",
    "macadamia nut", "mango", "maple syrup", "melon", "mint", "molasses", "monterey jack", "mozzarella", "mushroom",
    "mussel", "mustard", "mustard greens", "nutmeg", "oat", "okra", "olive", "onion", "orange",
    "oregano", "orzo", "oyster", "parmesan", "parsley", "parsnip", "pea", "peach", "peanut", "peanut butter",
    "pear", "pecan", "pepper", "persimmon", "pine nut", "pineapple", "pistachio", "plantain", "plum", "pomegranate",
    "pork", "potato", "prune", "pumpkin", "quince", "quinoa", "radicchio", "radish", "raisin", "raspberry",
    "rhubarb", "rice", "ricotta", "root vegetable", "rosemary", "rye",
    "sage", "salmon", "sardine", "sausage", "scallop", "sesame",
    "sesame oil", "shallot", "shellfish", "shrimp", "snapper", "soy", "spinach", "squash", "squid", "strawberry",
    "sugar snap pea", "sweet potato/yam", "tilapia", "tofu", "tomatillo", "tomato", "tree nut", "turnip",
    "vanilla", "veal", "vinegar", "wasabi", "watercress", "watermelon", "wild rice", "yogurt", "yuca",
    "zucchini", "marshmallow"
]

In [3]:
ingredients=pd.Series(ingredients)

In [7]:
ingredients.to_csv("data/ingredients.csv", index=False, header=False)

In [33]:
api_key = "8nQnIXeIWHj5iIe1tvb10mf275UZa5Hic1KLVAvV"
nutrition_url = "https://api.nal.usda.gov/fdc/v1/foods/search"

In [128]:
ingredients=ingredients[1:]

In [116]:
DV=pd.read_csv("data/DailyValuesNutrient.csv")


## 🍎 Сбор данных о пищевой ценности ингредиентов через **USDA FoodData Central API**

🔹 Используем **USDA FoodData Central API** для запроса по каждому ингредиенту из списка.

🔹 Для каждого ингредиента:

- Получаем **его питательные вещества (`foodNutrients`)** через API.
- Находим **`nutrientId`** нутриентов и сопоставляем с таблицей **дневных норм потребления (DV)**.
- Рассчитываем **% от дневной нормы:**

$$
\%DV = \frac{\text{nutrient value}}{\text{daily value}} \times 100
$$

- Сохраняем данные в **CSV**

In [130]:
nutrition_data = []

for ingr in tqdm(ingredients):
    if "/" in ingr:
        ingr_req = ingr.split("/")
    else:
        ingr_req = [ingr]

    for ingredient in ingr_req:
        response = requests.get(nutrition_url, params={
            "query": ingredient,
            "api_key": api_key,
            "pageSize": 1
        })
        if response.status_code == 200:
            nutrition_info = response.json()
            if nutrition_info['totalHits'] == 0:
                continue
            nutrients = nutrition_info["foods"][0]["foodNutrients"]
            nutrient_dict = {"ingredient": ingredient}
            for nutrient in nutrients:
                for name,dailyValue,id in DV.itertuples(index=False):
                    if nutrient['nutrientId'] == id:
                        nutrient_dict[name] = round((nutrient['value']/dailyValue)*100)
                        break
            nutrition_data.append(nutrient_dict)

  0%|          | 0/180 [00:00<?, ?it/s]

100%|██████████| 180/180 [03:47<00:00,  1.27s/it]


In [138]:
nutrition_df = pd.DataFrame(nutrition_data)
nutrition_df.to_csv("data/ingredient_nutrition_dv.csv", index=False)

In [3]:
recipes=pd.read_csv('data/epi_r.csv', usecols=[0])
recipes

,title
0,"Lentil, Apple, and Turkey Wrap"
1,Boudin Blanc Terrine with Red Onion Confit
2,Potato and Fennel Soup Hodge
3,Mahi-Mahi in Tomato Olive Sauce
4,Spinach Noodle Casserole
...,...
20047,Parmesan Puffs
20048,Artichoke and Parmesan Risotto
20049,Turkey Cream Puff Pie
20050,Snapper on Angel Hair with Citrus Cream


In [4]:
recipes=recipes['title']

In [22]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

recipes_data = []

In [19]:
for query in tqdm(recipes[0:1]):
    search_url = f"https://www.epicurious.com/search?q={query.replace(' ', '+')}"
    response = requests.get(search_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch: {query}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')

    # Найти первую ссылку на рецепт
    first_result = soup.find('div', class_='ClampContent-hilPkr fvKowN')
    if not first_result:
        continue

    a_tag = first_result.find('a', href=True)
    if not a_tag:
        continue

    url = "https://www.epicurious.com" + a_tag['href']
    title = a_tag.get_text(strip=True)

    # Переход на страницу рецепта
    recipe_resp = requests.get(url, headers=headers)
    if recipe_resp.status_code != 200:
        rating = "N/A"
        description = "N/A"
    else:
        recipe_soup = BeautifulSoup(recipe_resp.text, 'html.parser')

        # Рейтинг
        rating_tag = recipe_soup.find("p", class_="RatingRating-btVmKd")
        rating = rating_tag.get_text(strip=True) if rating_tag else "N/A"

        # Краткое описание
        tip_block = recipe_soup.find("div", {"data-testid": "cn-wrapper"})
        if tip_block:
            desc_p = tip_block.find("p")
            description = desc_p.get_text(strip=True) if desc_p else "N/A"
        else:
            description = "N/A"


    recipes_data.append({
        "searched_query": query,
        "title": title,
        "rating": rating,
        "description": description,
        "url": url
    })

    time.sleep(0.5)  # минимальная пауза на всякий случай

# Сохранение в DataFrame


100%|██████████| 1/1 [00:03<00:00,  3.27s/it]


In [21]:
df = pd.DataFrame(recipes_data)
df

,searched_query,title,rating,description,url
0,"Lentil, Apple, and Turkey Wrap","Lentil, Apple, and Turkey Wrap",2.5,"Cooking time for lentils can vary, depending o...",https://www.epicurious.com/recipes/food/views/...


In [ ]:
df.to_csv("data/recipes_with_urls.csv", index=False)

In [26]:
nest_asyncio.apply()

headers = {"User-Agent": "Mozilla/5.0"}
missed = []

def simplify_query(query):
    return query.replace(",", "").replace("and", "").lower().strip()

async def fetch_recipe(session, query):
    simplified_query = simplify_query(query)

    try:
        search_url = f"https://www.epicurious.com/search?q={simplified_query.replace(' ', '+')}"
        async with session.get(search_url, headers=headers) as resp:
            if resp.status != 200:
                print(f"[✗] Failed to fetch search for: {query}")
                missed.append(query)
                return None

            html = await resp.text()
            soup = BeautifulSoup(html, 'html.parser')

            # Найти первую ссылку
            first_result = soup.find('div', class_='ClampContent-hilPkr fvKowN')
            if not first_result:
                print(f"[!] No result found for: {query}")
                missed.append(query)
                return None

            a_tag = first_result.find('a', href=True)
            if not a_tag:
                print(f"[!] No link in result for: {query}")
                missed.append(query)
                return None

            url = "https://www.epicurious.com" + a_tag['href']
            title = a_tag.get_text(strip=True)

        # Step 2: Перейти на страницу рецепта
        async with session.get(url, headers=headers) as recipe_resp:
            if recipe_resp.status != 200:
                print(f"[!] Failed to fetch recipe page: {url}")
                return {
                    "searched_query": query,
                    "title": title,
                    "rating": "N/A",
                    "description": "N/A",
                    "url": url
                }

            html = await recipe_resp.text()
            recipe_soup = BeautifulSoup(html, 'html.parser')

            # Рейтинг
            rating_tag = recipe_soup.find("p", class_="RatingRating-btVmKd")
            rating = rating_tag.get_text(strip=True) if rating_tag else "N/A"

            # Описание
            tip_block = recipe_soup.find("div", {"data-testid": "cn-wrapper"})
            if tip_block:
                desc_p = tip_block.find("p")
                description = desc_p.get_text(strip=True) if desc_p else "N/A"
            else:
                description = "N/A"

        return {
            "searched_query": query,
            "title": title,
            "rating": rating,
            "description": description,
            "url": url
        }

    except Exception as e:
        print(f"[ERROR] {query}: {e}")
        missed.append(query)
        return None

async def scrape_all(recipes):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_recipe(session, query) for query in recipes]
        results = await tqdm_asyncio.gather(*tasks)
        return [r for r in results if r]

def run_scraper(recipes, output_file="data/recipes_with_urls.csv"):
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(scrape_all(recipes))
    df = pd.DataFrame(results)

    # Дозапись в CSV, если файл уже существует
    if os.path.exists(output_file):
        old_df = pd.read_csv(output_file)
        df = pd.concat([old_df, df], ignore_index=True)

    df.to_csv(output_file, index=False)
    print(f"[✓] Сохранено в {output_file} ({len(df)} рецептов всего)")

    # Сохраняем пропущенные
    if missed:
        with open("missed_queries.txt", "w", encoding="utf-8") as f:
            f.write("\n".join(missed))
        print(f"[!] Пропущено {len(missed)} рецептов. Сохранено в missed_queries.txt")

In [ ]:
run_scraper(recipes, "data/recipes_with_urls2.csv")

In [4]:
with open("missed_queries2.txt", "r", encoding="utf-8") as f:
    missed_queries = [line.strip() for line in f if line.strip()]

In [5]:
run_scraper(missed_queries[1000:], "data/recipes_with_urls2.csv")

  3%|▎         | 387/13520 [01:09<46:44,  4.68it/s]  

[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe


  3%|▎         | 472/13520 [01:20<16:52, 12.89it/s]  

[✗] Failed to fetch search for: Harissa Shrimp And Summer Vegetable Sauté
[✗] Failed to fetch search for: Dark Chocolate Avocado Brownies
[✗] Failed to fetch search for: Classic Turkey Stock


  4%|▎         | 480/13520 [01:20<14:28, 15.01it/s]

...

[ERROR] Grill-Roasted Rack of Lamb in Red Mole: 
[ERROR] Veal Scallops with Parsley, Lemon and Garlic: 
[ERROR] Heirloom Tomato Salad with Mozzarella and Basil: 
[ERROR] Crudites and Grilled Sausage with Sweet and Hot Chutneys: 
[ERROR] Tiny Baked Potatoes with Chervil Cream: 
[ERROR] Peanut Noodles with Shrimp: 
[ERROR] Sangria: 
[ERROR] Red Bell Pepper Corn Bread: 
[ERROR] Herb-Rubbed Turkey with Roasted-Garlic Gravy: 
[ERROR] Artichokes, Capers, Olives, Lemon Zest, and Italian Tuna on Pasta Shells: 
[ERROR] Tomato-Anchovy Pasta: 
[ERROR] Steamed Mussels with Sausages and Fennel: 
[ERROR] Sausage, Cranberry, and Pecan Stuffing: 
[ERROR] Ricotta-Filled Ravioli (Ravioli di Ricotta): 
[ERROR] Tiny Choux Puffs: 
[ERROR] Peach, Strawberry, and Banana Bruschetta: 
[ERROR] Mascarpone Tart with Honey, Oranges, and Pistachios: 
[ERROR] Lemongrass-Shallot Sambal: 
[ERROR] Polenta Budino With Plum Marmellata: 
[ERROR] Kentucky Bourbon Burgoo: 
[ERROR] Pan-Fried Pork Chops: 
[ERROR] Pepper, Rose

 97%|█████████▋| 13145/13520 [05:05<00:00, 2802.43it/s]

[ERROR] Blueberry Banana Shake: 
[ERROR] Delicata Squash and Roasted Mushrooms with Thyme: 
[ERROR] Grilled Lobster Salad with Avocado and Papaya: 
[ERROR] Baked Ziti with Mushrooms, Peppers, and Parmesan: 
[ERROR] Aunt Holly's Banana Bread: 
[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe


100%|██████████| 13520/13520 [05:41<00:00, 39.57it/s]  


[✓] Сохранено в data/recipes_with_urls2.csv (11106 рецептов всего)
[!] Пропущено 11228 рецептов. Сохранено в missed_queries.txt


In [8]:
with open("missed_queries3.txt", "r", encoding="utf-8") as f:
    missed_queries = [line.strip() for line in f if line.strip()]

In [9]:
run_scraper(missed_queries, "data/recipes_with_urls2.csv")

  3%|▎         | 291/11228 [00:53<20:28,  8.90it/s] 

[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe


 17%|█▋        | 1869/11228 [04:59<38:47,  4.02it/s]  

[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe


 17%|█▋        | 1871/11228 [05:00<58:38,  2.66it/s]

...

 98%|█████████▊| 11022/11228 [05:24<00:00, 280.01it/s] 

[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe
[ERROR] Steamed Mussels with Orange, Fennel, and Garlic: 
[ERROR] Prune Armagnac Sorbet: 
[ERROR] Linguine with Tomatoes, Olives, Feta, and Parsley: 
[ERROR] Sauteed Turkey Cutlets with Cranberry Orange Glaze: 
[ERROR] Lemon Cream Tart: 


100%|██████████| 11228/11228 [05:56<00:00, 31.50it/s] 

[ERROR] Cucumber and Radish Salad: 


[✓] Сохранено в data/recipes_with_urls2.csv (13434 рецептов всего)
[!] Пропущено 8900 рецептов. Сохранено в missed_queries.txt


In [11]:
with open("missed_queries4.txt", "r", encoding="utf-8") as f:
    missed_queries = [line.strip() for line in f if line.strip()]

In [13]:
run_scraper(missed_queries, "data/recipes_with_urls2.csv")

  3%|▎         | 270/8900 [00:52<16:01,  8.97it/s] 

[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe


  4%|▍         | 381/8900 [01:11<32:58,  4.31it/s]

[✗] Failed to fetch search for: Classic Date Bars


  4%|▍         | 387/8900 [01:11<26:28,  5.36it/s]

...

 99%|█████████▉| 8829/8900 [05:41<00:00, 86.80it/s] 

[ERROR] Milk-Braised Pork: 


100%|██████████| 8900/8900 [05:53<00:00, 25.18it/s]

[ERROR] Cheddar Chutney Toasts: 


[✓] Сохранено в data/recipes_with_urls2.csv (15822 рецептов всего)
[!] Пропущено 6512 рецептов. Сохранено в missed_queries.txt


In [17]:
with open("missed_queries5.txt", "r", encoding="utf-8") as f:
    missed_queries = [line.strip() for line in f if line.strip()]

In [18]:
run_scraper(missed_queries, "data/recipes_with_urls2.csv")

  9%|▉         | 584/6512 [01:37<15:39,  6.31it/s] 

[✗] Failed to fetch search for: Tomatillo Salsa


  9%|▉         | 595/6512 [01:38<10:44,  9.19it/s]

[✗] Failed to fetch search for: Tomatillo Salsa


 11%|█         | 697/6512 [01:52<12:39,  7.66it/s]

...

[ERROR] Grilled Seafood Salad Niçoise: 
[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe
[ERROR] Sticky Toffee Pudding with Blood Orange, Tangerine, and Whipped Crème Fraîche: 


 97%|█████████▋| 6286/6512 [05:14<00:01, 219.21it/s] 

[ERROR] Caramelized-Shallot Mashed Potatoes: 
[ERROR] Goan Curried Clams: 
[ERROR] Mint Julep Pineapple: 
[ERROR] Roast Beef, Red Pepper, Spinach, and Feta Sandwiches with Tahini Dressing: 
[ERROR] Potato, Apple, and Prune Stuffing: 
[ERROR] Garlic-Miso Pork Chops with Orange Bell Pepper and Arugula: 
[ERROR] Banana-Oatmeal Bars with Chocolate Chunks: 
[ERROR] Shrimp Toast with Pickled Ginger: 


100%|██████████| 6512/6512 [05:47<00:00, 18.74it/s] 


[✓] Сохранено в data/recipes_with_urls2.csv (18073 рецептов всего)
[!] Пропущено 4261 рецептов. Сохранено в missed_queries.txt


In [20]:
with open("missed_queries6.txt", "r", encoding="utf-8") as f:
    missed_queries = [line.strip() for line in f if line.strip()]

In [21]:
run_scraper(missed_queries, "data/recipes_with_urls2.csv")

  9%|▉         | 394/4261 [01:10<09:41,  6.65it/s]

[✗] Failed to fetch search for: Scallops, Okra, and Tomatoes in Coconut Curry Sauce


 15%|█▌        | 652/4261 [01:51<06:27,  9.31it/s]

[✗] Failed to fetch search for: Pecan Praline


 17%|█▋        | 738/4261 [02:08<59:10,  1.01s/it]

...

[ERROR] Malt-Beer-Brined Turkey with Malt Glaze: 
[ERROR] Pesce Per Due: 
[ERROR] Pear and Fig Pie with Hazelnut Crust: 
[ERROR] Imperial Hotel Fizz: 
[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe
[ERROR] Sugar Snap Peas and Pasta: 
[ERROR] Pine Nut Cardamom Scones: 
[ERROR] Chicken Salad with Rosemary, Almonds, and Green Onions: 


 97%|█████████▋| 4121/4261 [05:27<00:01, 75.25it/s] 

[ERROR] Guinness BBQ Sauce: 
[ERROR] Roasted Pears with Hazelnut Syrup and Candied Hazelnuts: 


100%|██████████| 4261/4261 [05:46<00:00, 12.28it/s]


[✓] Сохранено в data/recipes_with_urls2.csv (20328 рецептов всего)
[!] Пропущено 2006 рецептов. Сохранено в missed_queries.txt


In [23]:
with open("missed_queries7.txt", "r", encoding="utf-8") as f:
    missed_queries = [line.strip() for line in f if line.strip()]

In [24]:
run_scraper(missed_queries, "data/recipes_with_urls2.csv")

 19%|█▉        | 389/2006 [01:19<04:47,  5.62it/s]

[!] Failed to fetch recipe page: https://www.epicurious.com/simple-syrup-368889-recipe


 23%|██▎       | 456/2006 [01:33<04:48,  5.37it/s]

[✗] Failed to fetch search for: Oatmeal, Fig, and Walnut Bars


 26%|██▌       | 518/2006 [01:39<03:06,  7.98it/s]

...

[✗] Failed to fetch search for: Watercress and Radish Salad


 96%|█████████▌| 1916/2006 [05:03<00:04, 19.53it/s]

[ERROR] Plums with Prosciutto, Goat Cheese, Baby Arugula, and Champagne Vinegar: 


100%|██████████| 2006/2006 [05:06<00:00,  6.54it/s]


[✓] Сохранено в data/recipes_with_urls2.csv (22329 рецептов всего)
[!] Пропущено 5 рецептов. Сохранено в missed_queries.txt


In [27]:
with open("missed_queries8.txt", "r", encoding="utf-8") as f:
    missed_queries = [line.strip() for line in f if line.strip()]

In [28]:
run_scraper(missed_queries, "data/recipes_with_urls2.csv")

100%|██████████| 5/5 [00:04<00:00,  1.20it/s]

[✓] Сохранено в data/recipes_with_urls2.csv (22334 рецептов всего)


In [29]:
df = pd.read_csv("data/recipes_with_urls2.csv")

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22334 entries, 0 to 22333
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   searched_query  22334 non-null  object 
 1   title           22334 non-null  object 
 2   rating          18413 non-null  float64
 3   description     1776 non-null   object 
 4   url             22334 non-null  object 
dtypes: float64(1), object(4)
memory usage: 872.6+ KB


In [31]:
df_unique = df.drop_duplicates(subset=["searched_query","title", "url"], keep="first")

In [32]:
df_unique.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18605 entries, 0 to 22333
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   searched_query  18605 non-null  object 
 1   title           18605 non-null  object 
 2   rating          15380 non-null  float64
 3   description     1501 non-null   object 
 4   url             18605 non-null  object 
dtypes: float64(1), object(4)
memory usage: 872.1+ KB


In [33]:
df_unique.to_csv("data/recipes_with_urls_cleaned.csv", index=False)

print(f"[✓] Удалено {len(df) - len(df_unique)} дубликатов")

[✓] Удалено 3729 дубликатов


In [ ]:
for query in tqdm(recipes[0:1]):
    search_url = f"https://www.epicurious.com/search?q={query.replace(' ', '+')}"
    response = requests.get(search_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to fetch: {query}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')

    # Найти первую ссылку на рецепт
    first_result = soup.find('div', class_='ClampContent-hilPkr fvKowN')
    if not first_result:
        continue

    a_tag = first_result.find('a', href=True)
    if not a_tag:
        continue

    url = "https://www.epicurious.com" + a_tag['href']
    title = a_tag.get_text(strip=True)

    # Переход на страницу рецепта
    recipe_resp = requests.get(url, headers=headers)
    if recipe_resp.status_code != 200:
        rating = "N/A"
        description = "N/A"
    else:
        recipe_soup = BeautifulSoup(recipe_resp.text, 'html.parser')

        # Рейтинг
        rating_tag = recipe_soup.find("p", class_="RatingRating-btVmKd")
        rating = rating_tag.get_text(strip=True) if rating_tag else "N/A"

        # Краткое описание
        tip_block = recipe_soup.find("div", {"data-testid": "cn-wrapper"})
        if tip_block:
            desc_p = tip_block.find("p")
            description = desc_p.get_text(strip=True) if desc_p else "N/A"
        else:
            description = "N/A"


    recipes_data.append({
        "searched_query": query,
        "title": title,
        "rating": rating,
        "description": description,
        "url": url
    })

    time.sleep(0.5)  # минимальная пауза на всякий случай

# Сохранение в DataFrame


100%|██████████| 1/1 [00:03<00:00,  3.27s/it]
